In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nertag/SOCC_negation.tsv
/kaggle/input/nertag/clean_data.csv
/kaggle/input/nertag/unnegated_File_Name.csv
/kaggle/input/nertag/un-negated_clean_data.csv
/kaggle/input/nertag/File_Name.csv
/kaggle/input/nertag/reannotated_val.csv
/kaggle/input/nertag/Heyday_validationdata.csv


## Changing the transformers and tokenizer versions

In [2]:
pip install tokenizers==0.8.0rc4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 7.1 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.18.0 requires tokenizers!=0.11.3,<0.13,>=0.11.1, but you have tokenizers 0.8.0rc4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install transformers==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.2/757.2 KB 2.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.18.0
    Uninstalling transformers-4.18.0:
      Successfully uninstalled transformers-4.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.9.3 requires transformers<4.19,>=4.1, but you have transformers 3.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import transformers
transformers.__version__

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


'3.0.1'

In [4]:
import tokenizers
tokenizers.__version__

'0.8.0.rc4'

### Imports

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification,AutoConfig

The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)
The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


The current process just got forked. Disabling parallelism to avoid deadlocks...
To disable this warning, please explicitly set TOKENIZERS_PARALLELISM=(true | false)


### Reading the SOCC data

In [6]:
negation_data_socc= pd.read_csv("../input/nertag/SOCC_negation.tsv", sep = "\t")

In [7]:
negation_data_socc

,Sentence_index,Token_index,Token,Negation
0,0,1,Europe,False
1,0,2,has,False
2,0,3,done,False
3,0,4,a,False
4,0,5,HUGE,False
...,...,...,...,...
54883,2595,48,",",False
54884,2595,49,none,False
54885,2595,50,",",False
54886,2595,51,",",False


In [8]:
negation_data_socc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54888 entries, 0 to 54887
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Sentence_index  54888 non-null  int64 
 1   Token_index     54888 non-null  int64 
 2   Token           54888 non-null  object
 3   Negation        54888 non-null  bool  
dtypes: bool(1), int64(2), object(1)
memory usage: 1.3+ MB


### Data Wrangling

In [9]:
negation_data_socc['Negation'] = negation_data_socc['Negation'].map({True: 'True', False: 'False'})

In [10]:
negation_data_socc['Sentence']= negation_data_socc[['Sentence_index','Token','Negation']].groupby(['Sentence_index'])['Token'].transform(lambda x: ' '.join(x))

In [11]:
negation_data_socc['Negation']= negation_data_socc[['Sentence_index','Token','Negation']].groupby(['Sentence_index'])['Negation'].transform(lambda x: ','.join(x))

In [12]:
negation_data_socc = negation_data_socc[['Sentence','Negation']]

In [13]:
negation_data_socc = negation_data_socc.drop_duplicates().reset_index(drop=True)

In [14]:
negation_data_socc

,Sentence,Negation
0,Europe has done a HUGE amount to help refugees...,"False,False,False,False,False,False,False,Fals..."
1,But that is not going to happen !,"False,False,False,False,True,True,True,False"
2,Hilary did not lose because she was a woman .,"False,False,False,True,True,True,True,True,Tru..."
3,She lost because the US had their version of a...,"False,False,False,False,False,False,False,Fals..."
4,A restless electorate wanted an outsider .,"False,False,False,False,False,False,False"
...,...,...
2573,We are going to tell them that plenty of other...,"False,False,False,False,False,False,False,Fals..."
2574,Some of them even ' third world countries ' . ...,"False,False,False,False,False,False,False,Fals..."
2575,"We are going to tell them , when they run for ...","False,False,False,False,False,False,False,Fals..."
2576,Or the job.And we are going to tell our sons t...,"False,False,False,False,False,False,False,Fals..."


### Tokenization

In [15]:
labels_to_ids = {'True':1,'False':0}
ids_to_labels = {1: 'True', 0: 'False'}

In [16]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [17]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.Sentence[index].strip().split()  
        word_labels = self.data.Negation[index].split(",") 

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             is_pretokenized=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [18]:
train_size = 0.8
train_dataset = negation_data_socc.sample(frac=train_size,random_state=200)
test_dataset = negation_data_socc.drop(train_dataset.index).reset_index(drop=True)
test_dataset.drop(test_dataset.index[[233, 234, 457]], inplace=True)
test_dataset = test_dataset.reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
train_dataset.drop(train_dataset.index[[573, 1506, 1603, 1700]], inplace=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(negation_data_socc.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (2578, 2)
TRAIN Dataset: (2058, 2)
TEST Dataset: (513, 2)


### Importing and wrangling the HeyDay validation set

In [19]:
external_valset= pd.read_csv("../input/nertag/un-negated_clean_data.csv")

In [20]:
external_valset

,Tokens,Tags,is_negative,sentence
0,I,O,False,0
1,am,O,False,0
2,looking,O,False,0
3,for,O,False,0
4,a,O,False,0
...,...,...,...,...
2345,30,B-SIZE,False,166
2346,inch,I-SIZE,False,166
2347,white,B-COLOUR,False,166
2348,desk,B-PRODUCT,False,166


In [21]:
external_valset["Sentence"]= external_valset[["Tokens","is_negative","sentence"]].groupby(["sentence"])["Tokens"].transform(lambda x: ' '.join(x))

In [22]:
external_valset['is_negative'] = external_valset['is_negative'].map({True: 'True', False: 'False'})

In [23]:
external_valset["Negation"]= external_valset[["Tokens","is_negative","sentence"]].groupby(["sentence"])["is_negative"].transform(lambda x: ','.join(x))

In [24]:
external_valset = external_valset[['Sentence','Negation']]

In [25]:
external_valset = external_valset.drop_duplicates().reset_index(drop=True)

In [26]:
external_valset

,Sentence,Negation
0,I am looking for a black gloss 33 inch firecla...,"False,False,False,False,False,False,False,Fals..."
1,Looking for pre workout Pump addict instead of...,"False,False,False,False,False,False,False,Fals..."
2,i need a 48 inch glass sliding goof and a show...,"False,False,False,False,False,False,False,Fals..."
3,"Hello, do any of your free standing tubs have ...","False,False,False,False,False,False,False,Fals..."
4,I'm looking for a 24 inch white mirror that is...,"False,False,False,False,False,False,False,Fals..."
...,...,...
162,What rectangular shower units are available?,"False,False,False,False,False,False"
163,I am looking for a 35 inch bathroom sink count...,"False,False,False,False,False,False,False,Fals..."
164,I'm looking f or a Black Matte bathtub double ...,"False,False,False,False,False,False,False,Fals..."
165,Need a 27 inch frameless shower door,"False,False,False,False,False,False,False"


In [27]:
final_validation_set = dataset(external_valset, tokenizer, MAX_LEN)

### Loading the BERT Model

In [28]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [29]:
valid_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }
valid_loader = DataLoader(final_validation_set, **valid_params)

In [30]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [31]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [32]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(0.6867, device='cuda:0', grad_fn=<NllLossBackward0>)

In [33]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 2])

In [34]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

### Training and fine-tuneing the BERT model using the SOCC dataset

In [35]:

def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [36]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 0.8718448281288147
Training loss per 100 training steps: 0.42529961459412435
Training loss per 100 training steps: 0.3253485243234654
Training loss per 100 training steps: 0.2539540249382477
Training loss per 100 training steps: 0.21394646742341672
Training loss per 100 training steps: 0.19553941463147237
Training loss epoch: 0.19205539854880857
Training accuracy epoch: 0.9280739279681268


In [38]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

### Validating the BERT model on the test split of the SOCC dataset

In [39]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.7898760437965393
Validation loss per 100 evaluation steps: 0.07695304542966648
Validation loss per 100 evaluation steps: 0.08311045853002924
Validation Loss: 0.07939997079224119
Validation Accuracy: 0.9742784034994237


In [40]:
# from seqeval.metrics import classification_report
from sklearn.metrics import f1_score, classification_report

print(classification_report(labels, predictions, labels = list(labels_to_ids.keys()) ))

              precision    recall  f1-score   support

        True       0.85      0.95      0.90      1402
       False       0.99      0.97      0.98      9234

    accuracy                           0.97     10636
   macro avg       0.92      0.96      0.94     10636
weighted avg       0.97      0.97      0.97     10636



### Validating the BERT model on the HeyDay Validation set.

In [41]:
labels, predictions = valid(model, valid_loader)

Validation loss per 100 evaluation steps: 0.44423457980155945
Validation Loss: 0.3378222560346503
Validation Accuracy: 0.8931383090178608


In [42]:
print(classification_report(labels, predictions, labels = list(labels_to_ids.keys()) ))

              precision    recall  f1-score   support

        True       0.31      0.72      0.43       145
       False       0.98      0.89      0.93      2205

    accuracy                           0.88      2350
   macro avg       0.64      0.81      0.68      2350
weighted avg       0.94      0.88      0.90      2350



### Writing into the CSV file and generating the final tags

In [43]:
valset= pd.read_csv("../input/nertag/Heyday_validationdata.csv")

In [44]:
valset

,Sentence_Index,Tokens,Predicted_NER_Tags
0,0,I,O
1,0,am,O
2,0,looking,O
3,0,for,O
4,0,a,O
...,...,...,...
2345,166,30,I-SIZE
2346,166,inch,I-SIZE
2347,166,white,I-COLOUR
2348,166,desk,I-PRODUCT


In [45]:
token_list = valset.Tokens.values.tolist()
sentence_list = valset.Sentence_Index.values.tolist()
NER_list = valset.Predicted_NER_Tags.values.tolist()

In [46]:
import csv
rows = zip(sentence_list,token_list,NER_list,predictions)
with open('Heyday_predvalidationdata.csv', "w") as f:
    writer = csv.writer(f)
    writer.writerow(('Sentence_Index','Tokens','Predicted_NER_Tags','Predicted_Negation'))
    for row in rows:
        writer.writerow(row)

In [58]:
final_valset= pd.read_csv("../input/nertag/Heyday_final_predvalidationdata.csv")

In [59]:
final_valset

,Sentence_Index,Tokens,Predicted_NER_Tags,Predicted_Negation
0,0,I,O,False
1,0,am,O,False
2,0,looking,O,False
3,0,for,O,False
4,0,a,O,False
...,...,...,...,...
2345,166,30,I-SIZE,False
2346,166,inch,I-SIZE,False
2347,166,white,I-COLOUR,False
2348,166,desk,I-PRODUCT,False


In [60]:
final_valset = final_valset.replace('I-','', regex=True)

In [61]:
def label_final (row):
   if row['Predicted_NER_Tags'] == 'O' :
      return 'O'
   if row['Predicted_NER_Tags'] != 'O' and row['Predicted_Negation'] == True :
      return "I-N-"+row['Predicted_NER_Tags']
   if row['Predicted_NER_Tags'] != 'O' and row['Predicted_Negation'] == False :
      return "I-"+row['Predicted_NER_Tags']

In [62]:
final_valset['final_label'] = final_valset.apply (lambda row: label_final(row), axis=1)

In [63]:
final_valset

,Sentence_Index,Tokens,Predicted_NER_Tags,Predicted_Negation,final_label
0,0,I,O,False,O
1,0,am,O,False,O
2,0,looking,O,False,O
3,0,for,O,False,O
4,0,a,O,False,O
...,...,...,...,...,...
2345,166,30,SIZE,False,I-SIZE
2346,166,inch,SIZE,False,I-SIZE
2347,166,white,COLOUR,False,I-COLOUR
2348,166,desk,PRODUCT,False,I-PRODUCT


### Validating the final results

In [64]:
clean_data = pd.read_csv("../input/nertag/clean_data.csv")

In [69]:
clean_data = clean_data.replace('B-','I-', regex=True)

In [70]:
clean_data

,Tokens,Tags,sentence
0,I,O,0
1,am,O,0
2,looking,O,0
3,for,O,0
4,a,O,0
...,...,...,...
2345,30,I-SIZE,166
2346,inch,I-SIZE,166
2347,white,I-COLOUR,166
2348,desk,I-PRODUCT,166


In [71]:
clean_list = clean_data.Tags.values.tolist()
pred_list = final_valset.final_label.values.tolist()

In [72]:
print(classification_report(clean_list, pred_list))

               precision    recall  f1-score   support

  I-ATTRIBUTE       0.65      0.72      0.68       148
     I-COLOUR       0.86      0.65      0.74        48
   I-MATERIAL       1.00      0.58      0.73        33
I-N-ATTRIBUTE       0.85      0.57      0.68        98
   I-N-COLOUR       1.00      0.25      0.40         8
 I-N-MATERIAL       0.50      0.25      0.33         4
    I-N-PRICE       0.00      0.00      0.00         3
  I-N-PRODUCT       0.45      0.48      0.47        21
    I-N-SHAPE       0.00      0.00      0.00         2
     I-N-SIZE       0.34      0.56      0.43        18
  I-N-TEXTURE       0.00      0.00      0.00         4
      I-PRICE       1.00      0.25      0.40        12
    I-PRODUCT       0.92      0.87      0.90       320
      I-SHAPE       1.00      0.44      0.62         9
       I-SIZE       0.90      0.85      0.88       166
    I-TEXTURE       1.00      0.12      0.22         8
            O       0.92      0.98      0.95      1448

     acc

/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [60]:
for i in zip(tokens,predictions):
    print(i)

('I', 'False')
('am', 'False')
('looking', 'False')
('for', 'False')
('a', 'False')
('black', 'False')
('gloss', 'False')
('33', 'False')
('inch', 'False')
('fireclay', 'False')
('apron', 'False')
('sink', 'False')
('Looking', 'False')
('for', 'False')
('pre', 'False')
('workout', 'False')
('Pump', 'False')
('addict', 'False')
('instead', 'False')
('of', 'False')
('Karbolyn', 'False')
('Hydrate', 'False')
('Wich', 'False')
('one', 'False')
('is', 'False')
('better?', 'False')
('i', 'False')
('need', 'False')
('a', 'False')
('48', 'False')
('inch', 'False')
('glass', 'False')
('sliding', 'False')
('goof', 'False')
('and', 'False')
('a', 'False')
('shower', 'False')
('pan', 'False')
('system', 'False')
('for', 'False')
('500.99', 'False')
('or', 'False')
('less', 'False')
('Hello,', 'False')
('do', 'False')
('any', 'False')
('of', 'False')
('your', 'False')
('free', 'False')
('standing', 'False')
('tubs', 'False')
('have', 'False')
('a', 'False')
('matte', 'False')
('white', 'False')
('f